# **Homework 2**

In [1]:
import gdown
import zipfile
import os

In [2]:
#Check if files exists in data folder
if os.path.exists('data/'):
    print('Files already downloaded.')

else:
    output_path = 'data.zip'
    file_id = '1KDN-rFCq9IDJ7_kNW5y5Co100KNpklz-'
    url = f'https://drive.google.com/uc?id={file_id}'
    # Download the zip file
    gdown.download(url, output_path, quiet=False)

    # Extract the contents of the zip file
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        zip_ref.extractall('data')

    # Remove the zip file
    os.remove(output_path)


Files already downloaded.


In [3]:
if(os.path.exists('test/') and os.path.exists('train')):
    print('Files already extracted')
else:
    print('Extracting the test.zip and train.zip files...')
    # Extract the test.zip file
    with zipfile.ZipFile('data/public/test.zip', 'r') as zip_ref:
        zip_ref.extractall()

    # Extract the train.zip file
    with zipfile.ZipFile('data/public/train.zip', 'r') as zip_ref:
        zip_ref.extractall()

    print('Done!')

Files already extracted


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# Step 1: Data Loading and Preprocessing
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(96),   # Randomly resize and crop the image
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust color
    transforms.RandomRotation(30),       # Randomly rotate the image
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Define data transformations for validation (you can modify these based on your needs)
val_transform = transforms.Compose([
    transforms.Resize(96),
    transforms.CenterCrop(96),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load your dataset (adjust this based on your dataset structure)
train_dataset = ImageFolder(root='train', transform=train_transform)
val_dataset = ImageFolder(root='test', transform=val_transform)

num_workers = 4

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=num_workers)


In [5]:

# Step 2: Model Definition

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 5)  # Assuming 5 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNNModel()



In [6]:
criterion = nn.CrossEntropyLoss()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion.to(device)

# Optimizer and Scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Optional learning rate scheduler


In [7]:

# Step 4: Model Training
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:  # Assuming you have a DataLoader for your training set
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()  # Optional: Adjust learning rate if using a scheduler


    # Print average loss per epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/20, Loss: 1.5023788809776306
Epoch 2/20, Loss: 1.4700150167942048
Epoch 3/20, Loss: 1.4531512451171875
Epoch 4/20, Loss: 1.440071872472763
Epoch 5/20, Loss: 1.4337249648571015
Epoch 6/20, Loss: 1.4275204575061797
Epoch 7/20, Loss: 1.422253966331482
Epoch 8/20, Loss: 1.4208712685108185
Epoch 9/20, Loss: 1.4108968913555144
Epoch 10/20, Loss: 1.4145456218719483
Epoch 11/20, Loss: 1.4141099858283996
Epoch 12/20, Loss: 1.4196968412399291
Epoch 13/20, Loss: 1.4120732688903808
Epoch 14/20, Loss: 1.414239648580551
Epoch 15/20, Loss: 1.4120290958881379
Epoch 16/20, Loss: 1.4122127139568328
Epoch 17/20, Loss: 1.412547664642334
Epoch 18/20, Loss: 1.4157285070419312
Epoch 19/20, Loss: 1.4121592450141907
Epoch 20/20, Loss: 1.4128001046180725


In [8]:
torch.save(model.state_dict(), 'MyCNN.pth')

In [9]:

# Step 5: Model Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy}')


# Repeat steps 2-5 with a different architecture for Approach B
# ...

# Provide analysis and comments
# ...

Validation Accuracy: 0.6711531465987632
